In [ ]:
###  This is for installing the hf transformer library  
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#this is for  single  mask predictions.
#just change the model name for xml-roberta
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-multilingual-uncased')
unmasker("क्या राजेश कोई काम [MASK] रहा है?")


Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/672M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/872k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

[{'score': 0.7961074709892273,
  'token': 15807,
  'token_str': 'कर',
  'sequence': 'कया राजश कोई काम कर रहा ह?'},
 {'score': 0.027193384245038033,
  'token': 22352,
  'token_str': 'करत',
  'sequence': 'कया राजश कोई काम करत रहा ह?'},
 {'score': 0.02681123837828636,
  'token': 26235,
  'token_str': 'करता',
  'sequence': 'कया राजश कोई काम करता रहा ह?'},
 {'score': 0.01726345531642437,
  'token': 13465,
  'token_str': 'हो',
  'sequence': 'कया राजश कोई काम हो रहा ह?'},
 {'score': 0.013510110788047314,
  'token': 58358,
  'token_str': 'बना',
  'sequence': 'कया राजश कोई काम बना रहा ह?'}]

In [ ]:
#Code for multiple mask prediction in bert-base-multiple-uncased.
#This entire pipeline is for Bert.
#For xml-roberta look at the below cells.
from transformers import AutoModel,AutoTokenizer
from transformers import BertForMaskedLM
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForMaskedLM.from_pretrained('bert-base-multilingual-cased')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def create_mask_inx_tensor(encoded_input):
  import torch
  mask_index_tensors=[]
  for val in range(len(encoded_input["input_ids"][0])):
    if encoded_input["input_ids"][0][val]==103:
      mask_index_tensors.append(torch.tensor([val],))
  return mask_index_tensors

In [ ]:
#print(mask_index_tensors)

[tensor([3]), tensor([8])]


In [ ]:
import torch
def generate_predn(mask_index,encoded_input):
  output = model(**encoded_input)
  logits = output.logits
  word_list=[]
  from torch.nn import functional as F
  softmax = F.softmax(logits, dim = -1)
  mask_word = softmax[0, mask_index, :]
  top_10 = torch.topk(mask_word, 10, dim = 1)[1][0]
  #print("predn for token indx:{}".format(mask_index))
  for token in top_10:
    word=tokenizer.decode([token])
    word_list.append(word)
  return word_list
    #print(word)

In [ ]:
f=open("/content/drive/MyDrive/sample_dataset-1/triple_mask_bert","r")

In [ ]:
for text in f:
  print("text:",text)
  predn_op=[]
  encoded_input = tokenizer(text, return_tensors='pt')
  mask_index_tensors=create_mask_inx_tensor(encoded_input)
  for mask_index in mask_index_tensors:
    predn_op=(generate_predn(mask_index,encoded_input))
    #print("for token_inx:{}".format(mask_index))
    print(predn_op)

text: जून [MASK] सितंबर [MASK] दक्षिण-पश्चिम मानसून [MASK] मौसम (वर्षा)

[',', '-', '(', ':', ';', '/', 'से', 'और', '.', '।']
[',', '-', ':', ')', ';', '(', 'के', '.', 'और', '।']
['##े', '##ी', 'के', '##ा', '##िक', 'का', '-', '##न', 'और', '##ीय']
text: अक्टूबर और नवंबर [MASK] मानसून [MASK] लौटने [MASK] मौसम (शरद)

['-', ':', 'के', 'में', '(', ',', 'जैसे', 'का', ';', '।']
['में', 'से', 'और', ')', 'के', '-', '##ों', 'पर', ',', 'को']
['का', 'के', 'वाला', ')', 'वाले', '.', '।', 'और', 'या', 'में']
text: गर्मी [MASK] मौसम [MASK] सूर्य [MASK] किरणें अधिकतर सीधी पड़ती हैं।

['और', 'के', ',', 'का', '-', 'से', '##य', 'तथा', 'व', 'एवं']
['और', 'में', ',', 'तथा', 'के', '-', 'से', 'पर', 'एवं', ':']
['की', 'के', 'और', '-', '[UNK]', 'या', 'का', ',', 'व', 'से']
text: भारत [MASK] जलवायु [MASK] मोटे तौर [MASK] मानसूनी जलवायु कहा जाता है।

['की', 'के', 'में', 'का', '##ी', 'राष्ट्रीय', 'पर', 'भारतीय', 'या', 'राज्य']
['को', ',', 'जिसे', 'है', '।', 'के', 'में', 'भारत', 'और', 'भी']
['पर', 'से', '##म', 'है', '

In [ ]:
###-----------------Pipeline for bert ends here--------------------------------------------
### xml-roberta pipeline in the below cells.

In [ ]:
#Code for xml-roberta starts here
from transformers import pipeline
#unmasker = pipeline('fill-mask', model='xlm-roberta-base')
#unmasker("क्या राजेश कोई काम <mask> रहा है?")

In [ ]:
'''f=open("/content/drive/MyDrive/sample_dataset-1/test-sent_double_mask-bert-1.txt")
for str1 in f:
  token_str=""
  pred_op=unmasker(str1)
  for val in range(len(pred_op)):
    token_str=token_str+","+pred_op[val]["token_str"]
  #print(str1+"\t"+token_str)'''

'f=open("/content/drive/MyDrive/sample_dataset-1/test-sent_double_mask-bert-1.txt")\nfor str1 in f:\n  token_str=""\n  pred_op=unmasker(str1)\n  for val in range(len(pred_op)):\n    token_str=token_str+","+pred_op[val]["token_str"]\n  #print(str1+"\t"+token_str)'

In [ ]:
from transformers import AutoModelForMaskedLM,AutoTokenizer
#from transformers import BertForMaskedLM
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
model = AutoModelForMaskedLM.from_pretrained('xlm-roberta-base')

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import codecs
import os,subprocess
import re
def getLines(file,splitby):
    with codecs.open(file, 'r', encoding="utf-8", errors='ignore') as filer:
                    data = filer.read()
                    data = re.sub('\t+', '\t', data)
    data = data.strip('\n')
    lines = data.split(splitby)
    return lines

In [ ]:
postp_dict = {0:[],1:[],2:[],3:[]}
for lines in getLines('/content/drive/MyDrive/sample_dataset-1/vibhakti_list.txt','\n'):
  lines = lines.strip(' ')
  split = lines.split(' ')
  for i in range(len(split)):
    if split[i] not in postp_dict[i]:
      postp_dict[i].append(split[i])
  
print(postp_dict)

{0: ['आगे', 'अलावा', 'अंदर', 'अनुरुप', 'अनुसार', 'अन्तर्गत', 'अन्दर', 'आसपास', 'बाहर', 'बाद', 'बीच', 'भीतर', 'चलते', 'एकसाथ', 'गिर्द', 'हेतु', 'जैसा', 'जैसे', 'जैसी', 'का', 'कारण', 'करीब', 'के', 'की', 'को', 'लायक', 'लिए', 'लिये', 'मे', 'में', 'नज़दीक', 'ने', 'नें', 'नीचे', 'निकट', 'नुमा', 'ओर', 'पहले', 'पर', 'परे', 'परिणामस्वरूप', 'पर्यंत', 'पास', 'पश्चात्', 'पे', 'पीछे', 'फॉर', 'पू.', 'पूर्व', 'रूप', 'सहारे', 'सहित', 'सामने', 'समय', 'साथ', 'साथसाथ', 'सदृश', 'से', 'स्वरुप', 'स्वरूप', 'ऊपर', 'उधर', 'ऊँचाई', 'वजह', 'वाला', 'वाले', 'वाली', 'वालों', 'तहत', 'तक', 'तरफ', 'दौरान', 'दूर', 'दूरदराज़', 'दूरदूर', 'दूरी', 'द्वारा', 'योग्य'], 1: ['के', 'की', 'पीछे', 'से', 'में', 'वाले', 'तक', 'अंदर', 'अतिरिक्त', 'लिए', 'ओर', 'आगे', 'अलावा', 'अंर्तगत', 'अंतर्गत', 'अनुकूल', 'अनुसार', 'अन्तर्गत', 'अन्दर', 'आसपास', 'आधार', 'अधीन', 'बहाने', 'बाहर', 'बजाय', 'बराबर', 'बारे', 'बावजूद', 'बाद', 'बीच', 'बीचोंबीच', 'भीतर', 'चलते', 'इर्द', 'जरिए', 'जरिये', 'कारण', 'करीब', 'खिलाफ', 'किनारे', 'क्रमानुसार', 'लगभग'

In [ ]:
postp_dict={0: ['आगे', 'अलावा', 'अंदर', 'अनुरुप', 'अनुसार', 'अन्तर्गत', 'अन्दर', 'आसपास', 'बाहर', 'बाद', 'बीच', 'भीतर', 'चलते', 'एकसाथ', 'गिर्द', 'हेतु', 'जैसा', 'जैसे', 'जैसी', 'का', 'कारण', 'करीब', 'के', 'की', 'को', 'लायक', 'लिए', 'लिये', 'मे', 'में', 'नज़दीक', 'ने', 'नें', 'नीचे', 'निकट', 'नुमा', 'ओर', 'पहले', 'पर', 'परे', 'परिणामस्वरूप', 'पर्यंत', 'पास', 'पश्चात्', 'पे', 'पीछे', 'फॉर', 'पू.', 'पूर्व', 'रूप', 'सहारे', 'सहित', 'सामने', 'समय', 'साथ', 'साथसाथ', 'सदृश', 'से', 'स्वरुप', 'स्वरूप', 'ऊपर', 'उधर', 'ऊँचाई', 'वजह', 'वाला', 'वाले', 'वाली', 'वालों', 'तहत', 'तक', 'तरफ', 'दौरान', 'दूर', 'दूरदराज़', 'दूरदूर', 'दूरी', 'द्वारा', 'योग्य'], 1: ['के', 'की', 'पीछे', 'से', 'में', 'वाले', 'तक', 'अंदर', 'अतिरिक्त', 'लिए', 'ओर', 'आगे', 'अलावा', 'अंर्तगत', 'अंतर्गत', 'अनुकूल', 'अनुसार', 'अन्तर्गत', 'अन्दर', 'आसपास', 'आधार', 'अधीन', 'बहाने', 'बाहर', 'बजाय', 'बराबर', 'बारे', 'बावजूद', 'बाद', 'बीच', 'बीचोंबीच', 'भीतर', 'चलते', 'इर्द', 'जरिए', 'जरिये', 'कारण', 'करीब', 'खिलाफ', 'किनारे', 'क्रमानुसार', 'लगभग', 'लिहाज', 'लिये', 'मध्य', 'माध्यम', 'मुकाबले', 'मुताबिक', 'नजदीक', 'नाते', 'नीचे', 'निकट', 'पहले', 'फलस्वरूप', 'परिणाम', 'परिणामस्वरूप', 'पास', 'पश्चात्', 'पश्चात', 'प्रति', 'पूर्व', 'रूप', 'सहारे', 'समान', 'समानांतर', 'सामने', 'समय', 'संबंध', 'समीप', 'समीपस्थ', 'साथ', 'साथसाथ', 'ऊपर', 'उपरान्त', 'विपरीत', 'विरुद्ध', 'विरूद्ध', 'तहत', 'तौर', 'दरम्यान', 'दौरान', 'द्वारा', 'योग्य', 'अपेक्षा', 'बजाए', 'भाँति', 'जगह', 'जैसी', 'वजह', 'तरह', 'तरफ', 'तुलना', 'दृष्टि', 'का', 'इतर', 'लेकर', 'दूर', 'पर', 'को'], 2: ['ओर', 'को', 'से', 'की', 'का', 'के', 'पर', 'में', 'गिर्द', 'स्वरूप', 'तक', 'साथ', 'मुकाबले', 'तरह', 'रूप', 'समक्ष', 'लिए'], 3: ['में']}

In [ ]:
def get_mask_vib_index(indices):
  prev_ix = indices[0]
  ix_list = [0]
  for ix in indices[1:]:
    if prev_ix + 1 == ix:
      ix_list.append(1)
    else:
      ix_list.append(0)
    prev_ix = ix
  return ix_list
get_mask_vib_index([1,4,5,7])

[0, 0, 1, 0]

In [ ]:
import sys
class item :
    sent = ''
    probability = 0
class pred_q :
    pr = [None] * (100000)
    size = -1
    @staticmethod
    def enqueue( sent,  probability) :
        pred_q.size += 1
        pred_q.pr[pred_q.size] = item()
        pred_q.pr[pred_q.size].sent = sent
        pred_q.pr[pred_q.size].probability = probability
         
    @staticmethod
    def  peek() :
        highestProbability = -sys.maxsize
        ind = -1
        i = 0
        while (i <= pred_q.size) :
            if(highestProbability < pred_q.pr[i].probability) :
                highestProbability = pred_q.pr[i].probability
                ind = i
            i += 1
        return ind
       
    @staticmethod
    def dequeue() :
        ind = pred_q.peek()
        i = ind
        while (i < pred_q.size) :
            pred_q.pr[i] = pred_q.pr[i + 1]
            i += 1
        pred_q.size -= 1
### This code takes as input a sentence
### Encodes it using the Tokenizer
### Stores the index of masked tensor calling the create_mask_inx_tensor() function
### Calls the generate predn o/p function to get predicted o/p and scores
import torch
import itertools
import numpy as np
def create_mask_inx_tensor(encoded_input):
  
  mask_index_tensors=[]
  for val in range(len(encoded_input["input_ids"][0])):
    if encoded_input["input_ids"][0][val]==250001:
      mask_index_tensors.append(torch.tensor([val],))
  return mask_index_tensors

#----------------------------------------------------------
def generate_predn(index,mask_index,encoded_input,postp_dict):
  output = model(**encoded_input)
  #print(output)
  logits = output.logits
  input_ids=encoded_input.input_ids
  #print(logits)
  word_list=[]
  from torch.nn import functional as F
  softmax = F.softmax(logits, dim = -1)
  mask_word = softmax[0, mask_index, :]
  #-----------------------------------------------
  #print(mask_word)
  values,predictions=mask_word.topk(5)
  result = []
  single_mask = values.shape[0] == 1
  for i, (_values, _predictions) in enumerate(zip(values.tolist(), predictions.tolist())):
    row=[]
    unk_row = []
    for v, p in zip(_values, _predictions):
                # Copy is important since we're going to modify this array in place
      tokens = input_ids.numpy().copy()
      proposition = {"score": v, "token": tokenizer.decode([p])}
      if proposition["token"] in postp_dict[index]:
        row.append(proposition)
      else:
        #print(proposition["token"])
        proposition["token"] += '_unk'
        unk_row.append(proposition)
    if not row:
      result.append(unk_row)
    else:
      result.append(row)
  #print(result)
  return result
  #-----------------------------------------------
  top_10 = torch.topk(mask_word, 10, dim = 1)[1][0]
  #print(type(top_10))
  #print(top_10)
  #print("predn for token indx:{}".format(mask_index))
  for token in top_10:
    word=tokenizer.decode([token])
    word_list.append(word)
  return word_list
#-------------------------------------------------
#text=""The world will end in <mask>""
text1="अंतिम प्रकार <mask> खोजबीन <mask> <mask> हम ये सब बातें कर रहे हैं"
text2 = "मुन्नालाल <mask> <mask> दो भैंस और कुछ मुर्गियाँ भी हैं ।"
text3 = "मृदा <mask> अधिक उपजाऊ बनाने <mask> <mask> अभी खाद डाला है ।"
text4 = "क्या आप अपने क्षेत्र <mask> पाए जाने <mask> पाँच पक्षियों <mask> नाम बता सकते हैं?"
text5 = "अनंत जन्मों <mask> जिस पूर्ण पुरुषोत्तम <mask> वे ढूँढते रहें वह इस जन्म <mask> उनके ही देह <mask> प्रगट हो गये"
def fill_mask(text):
  try:
    text = text.replace('<mask><mask>','<mask> <mask>')
    text = text.replace('  ',' ')
    mtoken_ind = [m.start() for m in re.finditer('<mask>', text)]
    vib_index = get_mask_vib_index(mtoken_ind)
    encoded_input = tokenizer(text, return_tensors='pt')
    mask_index_tensors=create_mask_inx_tensor(encoded_input)
  #print(encoded_input)
  #print("text:",text)
    predn_op=[]
  #print(mask_index_tensors)
    for index in range(len(mask_index_tensors)):
      predn_op.append((generate_predn(vib_index[index],mask_index_tensors[index],encoded_input,postp_dict)))
    #print(predn_op[-1])
    #print("for token_inx:{}".format(mask_index))'
    top_3_pred = []
    #if len(predn_op) == 1:
      #top_3_pred = predn_op[0][0][0]["token"]
    all_list = [l[0] for l in predn_op]
    permutations = list(itertools.product(*all_list))
    for ps in permutations:
      tokens = [p["token"] for p in ps]
      scores = [p["score"] for p in ps]
      sent = ' '.join(tokens)
      prob = np.prod(scores)
      #print([sent,prob])
      pred_q.enqueue(sent,prob)
    
    
    for j in range(3):
      text0 = text
      pred_masks = pred_q.pr[pred_q.peek()].sent.split(' ')
      length = len(pred_masks)
      #print([pred_masks,predn_op])
      for i in range(length):
        text0 = text0.replace('<mask>',pred_masks[i],1)
      top_3_pred.append([text0,pred_q.pr[pred_q.peek()].probability])
      pred_q.dequeue()
      
    return top_3_pred
  except:
    print(f"failed for {text}")
    return []
fill_mask(text5)


[['अनंत जन्मों से जिस पूर्ण पुरुषोत्तम को वे ढूँढते रहें वह इस जन्म में उनके ही देह में प्रगट हो गये',
  0.07566140207825515],
 ['अनंत जन्मों से जिस पूर्ण पुरुषोत्तम को वे ढूँढते रहें वह इस जन्म से उनके ही देह में प्रगट हो गये',
  0.04228592655372625],
 ['अनंत जन्मों से जिस पूर्ण पुरुषोत्तम को वे ढूँढते रहें वह इस जन्म में उनके ही देह से प्रगट हो गये',
  0.028076562916919245]]

In [ ]:
def get_pred_col(sent):
  #sent = row['masked']
  if '<mask>' in sent:
    pred_top_3 = fill_mask(sent)
    pred_top_3_sent = [i[0] for i in pred_top_3]
    return ';'.join(pred_top_3_sent)
  else:
    return sent
get_pred_col("प्रत्येक जीव <mask> शिष्यपद ग्रहण करने <mask> दृष्टि जिसने पाई है वही ज्ञानी हो सकता है")
#get_pred_col("ज्ञानी पुरुष <mask> भक्ति <mask> कीर्तन भक्ति करना तो सर्वश्रेष्ठ भक्ति है")

'प्रत्येक जीव का शिष्यपद ग्रहण करने की दृष्टि जिसने पाई है वही ज्ञानी हो सकता है;प्रत्येक जीव को शिष्यपद ग्रहण करने की दृष्टि जिसने पाई है वही ज्ञानी हो सकता है;प्रत्येक जीव के शिष्यपद ग्रहण करने की दृष्टि जिसने पाई है वही ज्ञानी हो सकता है'

In [ ]:
import pandas as pd
masked_ds_findf = pd.read_csv('/content/drive/My Drive/LTRC/masked_ds_final',sep = '\t',header=None)
masked_ds_findf.columns = [['masked','original']]
masked_ds_findf.head()
masked_ds_findf = masked_ds_findf

In [ ]:
#masked_ds_findf['predictions'] = masked_ds_findf.apply(lambda x : get_pred_col(x),axis=1)
#masked_ds_findf['prediction2'] = masked_ds_findf.apply(lambda x : fill_mask(x['masked']),axis=1)
#masked_ds_findf['prediction3'] = masked_ds_findf.apply(lambda x : fill_mask(x['masked']),axis=1)
#masked_ds_findf

failed for परम पूज्य दादाश्री हिन्दी <mask> बहुत कम बोलते थे कभी हिन्दी भाषी लोगों <mask> आने <mask> जो गुजराती नहीं समझ पाते थे उनके <mask> पूज्यश्री हिन्दी बोल लेते थे उसी वाणी <mask> कैसेट्स <mask> <mask> ट्रान्स्क्राइब करके यह आप्तवाणी ग्रंथ बना है
failed for उसी आप्तवाणी ग्रंथ <mask> फिर से संकलित करके यह सात लघु पुस्तिकायें बनायी गयी हैं
failed for उनकी हिन्दी प्यॉर हिन्दी नहीं है फिर भी सुननेवाले <mask> उनका अंतर आशय एक्झेक्ट समझ <mask> आ जाता है
failed for उनकी वाणी हृदयस्पर्शी होने <mask> <mask> <mask> निकली वैसी ही संकलित करके प्रस्तुत की गई है ताकि सुज्ञ पाठक <mask> उनके डाइरेक्ट शब्द पहुँचे
failed for उनकी हिन्दी याने गुजराती अंग्रेजी और हिन्दी <mask> मिश्रण
failed for फिर भी सुनने <mask> पढऩे <mask> बहुत मीठी लगती है नेचुरल लगती है जीवित लगती है
failed for जो शब्द हैं वह भाषाकीय दृष्टि <mask> सीधे सादे हैं किन्तु ज्ञानी पुरुष <mask> दर्शन निरावरण होता है इसलिए उनके प्रत्येक वचन आशयपूर्ण मार्मिक मौलिक और सामनेवाले <mask> व्यू पोइंट <mask> एक्झैक्ट समझकर निकलने <mask> <mask>

In [ ]:
masked_ds_findf

,masked,original,predictions
0,आपश्री <mask> अपने जीवनकाल <mask> ही पूज्य डॉ,आपश्री ने अपने जीवनकाल में ही पूज्य डॉ,आपश्री <mask> अपने जीवनकाल <mask> ही पूज्य डॉ
1,नीरूबहन अमीन नीरूमाँ <mask> आत्मज्ञान प्राप्त ...,नीरूबहन अमीन नीरूमाँ को आत्मज्ञान प्राप्त करवा...,नीरूबहन अमीन नीरूमाँ <mask> आत्मज्ञान प्राप्त ...
2,दादाश्री <mask> देहविलय पश्चात् नीरूमाँ उसी प्...,दादाश्री के देहविलय पश्चात् नीरूमाँ उसी प्रकार...,दादाश्री <mask> देहविलय पश्चात् नीरूमाँ उसी प्...
3,पूज्य दीपकभाई देसाई <mask> दादाश्री <mask> सत्...,पूज्य दीपकभाई देसाई को दादाश्री ने सत्संग करने...,पूज्य दीपकभाई देसाई <mask> दादाश्री <mask> सत्...
4,नीरूमाँ <mask> उपस्थिति <mask> ही उनके आशीर्वा...,नीरूमाँ की उपस्थिति में ही उनके आशीर्वाद से पू...,नीरूमाँ <mask> उपस्थिति <mask> ही उनके आशीर्वा...
...,...,...,...
3701,<mask> वो बड़ा कम्प्यूटर होता है न वैसा ही उसका...,जैसे वो बड़ा कम्प्यूटर होता है न वैसा ही उसका क...,<mask> वो बड़ा कम्प्यूटर होता है न वैसा ही उसका...
3702,प्रश्नकर्ता ये कम्प्यूटर <mask> बात समझ <mask>...,प्रश्नकर्ता ये कम्प्यूटर की बात समझ में नहीं आई,प्रश्नकर्ता ये कम्प्यूटर <mask> बात समझ <mask>...
3703,दादाश्री समझ लो तुमने विचार किया कि मंदिर बनान...,दादाश्री समझ लो तुमने विचार किया कि मंदिर बनान...,दादाश्री समझ लो तुमने विचार किया कि मंदिर बनान...
3704,इधर तुमने ये विचार किया वो विचार बड़ा कम्प्यूटर...,इधर तुमने ये विचार किया वो विचार बड़ा कम्प्यूटर...,इधर तुमने ये विचार किया वो विचार बड़ा कम्प्यूटर...


In [ ]:
if len(predn_op) == 2:
    for i in predn_op[0][0]:
      for j in predn_op[1][0]:
        sent = ' '.join([i["token"],j["token"]])
        prob = i["score"]*j["score"]
        pred_q.enqueue(sent,prob)
  elif len(predn_op) == 3:
    for i in predn_op[0][0]:
      for j in predn_op[1][0]:
        for k in predn_op[2][0]:
          sent = ' '.join([i["token"],j["token"],k["token"]])
          prob = i["score"]*j["score"]*k["score"]
          pred_q.enqueue(sent,prob)

l = len(predn_op)
    if len(predn_op) == 2:
      pred_masks = pred_q.pr[pred_q.peek()].sent.split(' ')
      for i in range(2):
        text0 = text0.replace('<mask>',pred_masks[i],1)
      top_3_pred.append([text0,pred_q.pr[pred_q.peek()].probability])
      pred_q.dequeue()
    elif len(predn_op) == 3:
      pred_masks = pred_q.pr[pred_q.peek()].sent.split(' ')
      for i in range(3):
        text0 = text0.replace('<mask>',pred_masks[i],1)
      top_3_pred.append([text0,pred_q.pr[pred_q.peek()].probability])
      pred_q.dequeue()

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained('roberta-base')
model = AutoModelForMaskedLM.from_pretrained('roberta-base')

sequence = f"The world will end in <mask>" # "The world will end in <mask>"

input_seq = tokenizer.encode(sequence, return_tensors='pt') # tensor([[0, 133, 232, 40, 253, 11, 50264, 2]])
mask_token_index = torch.where(input_seq == tokenizer.mask_token_id)[1] # (tensor([0]), tensor([6])) - we only want the the 2nd dimension

token_logits = model(input_seq).logits
#probs = logits.softmax(dim=0)
masked_token_logits = token_logits[0, mask_token_index, :]
probs = masked_token_logits.softmax(dim=0)
top_5_tokens = torch.topk(masked_token_logits, 5, dim=1).indices[0].tolist()

print('sequence:', sequence)
print('input_seq:', input_seq)
print('mask_token_index:', mask_token_index)
print('token_logits:', token_logits)
print('masked_token_logits:', masked_token_logits)
print("probs:",probs)
print('top_5_tokens:', top_5_tokens)
#values = probs[..., target_inds]
#sort_inds = list(reversed(values.argsort(dim=-1)))
#values = values[..., sort_inds]
#predictions = target_inds[sort_inds]

sequence: The world will end in <mask>
input_seq: tensor([[    0,   133,   232,    40,   253,    11, 50264,     2]])
mask_token_index: tensor([6])
token_logits: tensor([[[ 3.5597e+01, -3.9925e+00,  2.3101e+01,  ...,  2.8275e+00,
           4.9039e+00,  1.2930e+01],
         [ 4.8462e+00, -2.8101e+00,  1.8639e+01,  ...,  1.7748e+00,
           5.6141e+00,  7.7843e+00],
         [-2.0385e+00, -3.8334e+00,  1.0115e+01,  ..., -1.2934e+00,
          -9.7073e-01,  1.1687e+00],
         ...,
         [-7.7622e-02, -3.6380e+00,  1.5823e+01,  ...,  1.0678e+00,
           8.4141e-01,  5.1897e+00],
         [-7.9398e-01, -3.1608e+00,  1.0495e+01,  ..., -3.2913e-01,
           3.3966e-01,  1.5996e+00],
         [ 1.4663e+01, -5.3171e+00,  2.4834e+01,  ..., -1.7567e+00,
          -2.6692e-02,  7.5554e+00]]], grad_fn=<ViewBackward0>)
masked_token_logits: tensor([[-0.7940, -3.1608, 10.4950,  ..., -0.3291,  0.3397,  1.5996]],
       grad_fn=<IndexBackward0>)
probs: tensor([[1., 1., 1.,  ..., 1., 1., 1

In [ ]:
from torch.nn import functional as F
softmax = F.softmax(masked_token_logits, dim = -1)

In [ ]:
mask_word = softmax[0, mask_token_index, :]


IndexError: ignored

In [ ]:
###----------------------Pipeline for xml-roberta ends here--------------------------------
###Below cells have the code for mlm using mt5 and extra ids.

In [ ]:
f=open("/content/drive/MyDrive/sample_dataset-1/triple_mask_xlm-roberta","r")

In [ ]:
for text in f:
  print("text:",text)
  predn_op=[]
  encoded_input = tokenizer(text, return_tensors='pt')
  mask_index_tensors=create_mask_inx_tensor(encoded_input)
  #print(mask_index_tensors)
  for mask_index in mask_index_tensors:
    predn_op=(generate_predn(mask_index,encoded_input))
    #print("for token_inx:{}".format(mask_index))
    print(predn_op)

NameError: ignored

In [ ]:
!pip install SentencePiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 4.7 MB/s 


In [ ]:

from transformers import MT5ForConditionalGeneration, T5Tokenizer

model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")
tokenizer = T5Tokenizer.from_pretrained("google/mt5-small")

'''input_ids = tokenizer("Ram goes <extra_id_0> School.", return_tensors="pt").input_ids

sequence_ids = model.generate(input_ids,num_beams=5, num_return_sequences=5)
sequences = tokenizer.batch_decode(sequence_ids)
sequences'''

Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

'input_ids = tokenizer("Ram goes <extra_id_0> School.", return_tensors="pt").input_ids\n\nsequence_ids = model.generate(input_ids,num_beams=5, num_return_sequences=5)\nsequences = tokenizer.batch_decode(sequence_ids)\nsequences'

In [ ]:
f=open("/content/drive/MyDrive/sample_dataset-1/test-sent-double_mask_consecutive_mt5.txt","r")

In [ ]:
for sent in f:
  input_ids = tokenizer(sent, return_tensors="pt").input_ids
  sequence_ids = model.generate(input_ids,num_beams=5, num_return_sequences=5)
  sequences = tokenizer.batch_decode(sequence_ids)
  print(sent)
  for op in sequences:
    print(op)

उदाहरण<extra_id_0> <extra_id_1> मौसम कभी गर्म या कभी ठंडा होता है

<pad> <extra_id_0>.</s><pad>
<pad> <extra_id_0> ></s><pad>
<pad> <extra_id_0>:</s><pad>
<pad> <extra_id_0>...</s><pad>
<pad> <extra_id_0>.</s>
दिन<extra_id_0> <extra_id_1> गर्म एवं शुष्क पवन बहती है जिसे लू कहा जाता है।

<pad> <extra_id_0>_id_0></s><pad>
<pad> <extra_id_0>_id__0></s>
<pad> <extra_id_0>:</s><pad><pad><pad><pad>
<pad> <extra_id_0>.</s><pad><pad><pad><pad>
<pad> <extra_id_0> :</s><pad><pad><pad>
उनको बचाने<extra_id_0> <extra_id_1> बहुत से नेशनल पार्क, पशुविहार तथा जीवमंडल आरक्षित क्षेत्र स्थापित किए गए हैं।

<pad> <extra_id_0> ।</s>
<pad> <extra_id_0>।</s><pad>
<pad> <extra_id_0> -</s>
<pad> <extra_id_0>.</s><pad>
<pad> <extra_id_0> :</s>
दक्षिण-पश्चिम मानसून<extra_id_0> <extra_id_1> आर्द्र पवनें कहाँ बहती हैं?

<pad> <extra_id_0>.</s><pad>
<pad> <extra_id_0> :</s>
<pad> <extra_id_0>:</s><pad>
<pad> <extra_id_0>।</s><pad>
<pad> <extra_id_0> ।</s>
अनुकूल स्थलाकृति, मृदा और जलवायु कृषि क्रियाकलाप<extra_id_0>